In [2]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv

dataset = 'Cora'
path = osp.join(osp.dirname(osp.realpath("./")), '..', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]


class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Net, self).__init__()

        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(8 * 8, out_channels, heads=1, concat=False,
                             dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Processing...
Done!
Epoch: 001, Train: 0.2857, Val: 0.2240, Test: 0.2290
Epoch: 002, Train: 0.2571, Val: 0.2080, Test: 0.2340
Epoch: 003, Train: 0.3571, Val: 0.2140, Test: 0.2380
Epoch: 004, Train: 0.5357, Val: 0.3440, Test: 0.3450
Epoch: 005, Train: 0.7714, Val: 0.5120, Test: 0.5290
Epoch: 006, Train: 0.8429, Val: 0.5700, Test: 0.6160
Epoch: 007, Train: 0.8357, Val: 0.5300, Test: 0.5820
Epoch: 008, Train: 0.8714, Val: 0.5660, Test: 0.6070
Epoch: 009, Train: 0.8714, Val: 0.5720, Test: 0.6180
Epoch: 010, Train: 0.8714, Val: 0.5700, Test: 0.5930
Epoch: 011, Train: 0.8857, Val: 0.5900, Test: 0.6100
Epoch: 012, Train: 0.8929, Val: 0.6360, Test: 0.6330
Epoch: 013, Train: 0.9143, Val: 0.6720, Test: 0.6650
Epoch: 014, Train: 0.9143, Val: 0.6820, Test: 0.6870
Epoch: 015, Train: 0.9071, Val: 0.6940, Test: 0.6910
Epoch: 016, Train: 0.9071, Val: 0.7040, Test: 0.6930
Epoch: 017, Train: 0.9286, Val: 0.7200, Test: 0.7060
Epoch: 018, Train: 0.9286, Val: 0.7220, Test: 0.7140
Epoch: 019, Train: 0.9357,

In [21]:
dataset = 'Cora'
path = osp.join(osp.dirname(osp.realpath("./")), '..', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
cora = dataset[0]

In [22]:
cora

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [7]:
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [9]:
from torch_geometric.utils import from_networkx, to_networkx
import networkx as nx

In [38]:
G = nx.barabasi_albert_graph(20, 2)

G.add_node(21)

In [39]:
data = from_networkx(G)
data.x = torch.tensor([[float(d)] for d in list(dict(G.degree()).values())])
data.y = torch.tensor([0 if d > 10 else 1 for d in list(dict(G.degree()).values())])


data.train_mask = torch.tensor([True if d < 5 else False for d in range(21)])
data.test_mask = torch.tensor([True if d > 5 and d < 10 else False for d in range(21)])
data.val_mask = torch.tensor([True if d > 10 else False for d in range(21)])

In [43]:
data.edge_index.type()

'torch.cuda.LongTensor'

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(1, 2).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 002, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 003, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 004, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 005, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 006, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 007, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 008, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 009, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 010, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 011, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 012, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 013, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 014, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 015, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 016, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 017, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 018, Train: 1.0000, Val: 1.0000, Test: 1.0000
Epoch: 019, Train: 1.0000, Val: 1.0000, Test: 

In [1]:
import numpy as np

In [2]:
a = np.array([i for i in range(100)])
print(np.average(a), np.std(a))
print(np.average(a*100), np.std(a*100))


49.5 28.86607004772212
4950.0 2886.607004772212
